### Main Process

In [ ]:
import yt
import glob
import os
from simesh.frontends.amrvac.datio import get_header

data_slice = './datas'
slice_list = glob.glob(os.path.join(data_slice, 'fs2d*.dat'))
slice_list.sort()

units = dict(length_unit=(1e9, 'cm'), temperature_unit=(1e6, 'K'),
            numberdensity_unit=(1e9, 'cm**-3'))

In [ ]:
tfile = glob.glob('./fs2d/data/quad1k2_d*.dat')
ds = yt.load(tfile[0], units_override=units, unit_system='cgs')
p = yt.plot_2d(ds, 'e', origin='native')
p.show()

In [ ]:
slice0 = slice_list[50]
# slice0 = './fs2d/data/fs2d0000.dat'
ds = yt.load(slice0, units_override=units, unit_system='cgs')
phys_vars = ['Te']
for var in phys_vars:
    p = yt.plot_2d(ds, var, origin='native')
    # p.set_log(var, False)
    # p.set_zlim(var, 1e-1, 1)
    # p.set_zlim(var, -10, 0)
    p.annotate_timestamp()
    # p.annotate_grids()
    p.show()

In [ ]:
for i in range(0,len(slice_list)):
    slice0 = slice_list[i]
    ds = yt.load(slice0, units_override=units, unit_system='cgs')
    phys_vars = ['rho']
    for var in phys_vars:
        p = yt.plot_2d(ds, var, origin='native')
        if var == 'rho':
            p.set_zlim(var, 1e-1, 1e0)
        p.annotate_timestamp()
        p.save('./figs/fs2d'+str(i).zfill(4)+'_'+var+'.png')
# ffmpeg -framerate 12 -pattern_type glob -i ./figs/'fs2d*_rho.png' -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p rho.mp4 

### Analysis on the cooling curve (Colgan_DM)

In [ ]:
import numpy as np

fiso = './fs2d/data/ciso.bin'

def read_ciso(filename):
    with open(filename, 'rb') as f:
        ncool = np.fromfile(f, dtype=np.int32, count=1)[0]
        tcool = np.fromfile(f, dtype=np.float64, count=ncool)
        Lcool = np.fromfile(f, dtype=np.float64, count=ncool)
        ciso1 = np.fromfile(f, dtype=np.float64, count=ncool)
    return tcool, Lcool, ciso1

In [ ]:
import matplotlib.pyplot as plt

tcool, Lcool, ciso1 = read_ciso(fiso)
plt.plot(tcool, Lcool)
plt.xlabel('T (1 MK)')
plt.ylabel(r'$\Lambda(T)\ (normalised)$')
plt.title('Cooling curve')
plt.loglog()
plt.show()

$$ C_{isobaric} = \rho (\frac{\partial L}{\partial T})_\rho - \frac{\rho^2}{T}(\frac{\partial L}{\partial \rho})_T + k^2 \kappa < 0$$
$$L = n_H n_e \Lambda(T)/\rho \ \text{, which \ is} \ \rho^{\prime 2} \Lambda^{\prime}(T^{\prime}) / \rho^{\prime} \ \text{in normalised unit}$$
$$C_{isobaric} = \rho^2(\frac{\partial \Lambda(T)}{\partial T}-\frac{\Lambda(T)}{T})+k^2 \kappa$$
> Xia et al. 2011

In [ ]:
# plt.plot(tcool, ciso1)
plt.xlabel('T (1 MK)')
plt.ylabel(r'$ - (\partial\Lambda(T)/\partial T-\Lambda(T)/T)\ (normalised)$')
plt.title('Cooling curve')
plt.xscale('log')
plt.yscale('log')
plt.ylim(1e-6, 11)
plt.plot(tcool[ciso1<0], -ciso1[ciso1<0])
plt.show()


### vtk to csv (lines to points for speical refinement)

In [ ]:
import pyvista as pv
import numpy as np
lines = pv.read('fs2d/data/tilt_lines.vtk')
# Save points to CSV file
points = lines.points
np.savetxt('fs2d/data/tilt_points.csv', points, header=str(points.shape[0]), comments='')

### 2D relaxation test

In [ ]:
import yt
import glob
import os
from simesh.frontends.amrvac.datio import get_header

data_slice = './datas/'
data_slice = './fs2d/data/'
slice_list = glob.glob(os.path.join(data_slice, 'bheat*.dat'))
slice_list.sort()

units = dict(length_unit=(1e9, 'cm'), temperature_unit=(1e6, 'K'),
            numberdensity_unit=(1e9, 'cm**-3'))
            

In [ ]:
slice0 = slice_list[-10]
# slice0 = './fs2d/data/fs2d0002.dat'
ds = yt.load(slice0, units_override=units, unit_system='cgs')
phys_vars = ['Te']
# def _temperature(field, data):
#     return 2.0/3.0*(data[('amrvac','e')]-1/2*data[('amrvac','m1')]**2/data['amrvac','rho']) / data[('amrvac','rho')]
# ds.add_field(("amrvac","Te"), function=_temperature, sampling_type='local', units="code_length**2/code_time**2")
for var in phys_vars:
    p = yt.plot_2d(ds, var, origin='native')
    # if var == 'm1':
    #     p.set_log(var, False)
    # p.set_log(var, False)
    # p.set_zlim(var, 1e-2, 8e-1)
    p.annotate_timestamp()
    p.annotate_grids()
    p.annotate_streamlines('b1','b2', density=2, color='white', linewidth=1.2)
    p.show()

In [ ]:
units = dict(length_unit=(1e9, 'cm'), temperature_unit=(1e6, 'K'),
            numberdensity_unit=(1e9, 'cm**-3'))

for file in slice_list:
    var = 'Te'
    ds = yt.load(file, units_override=units, unit_system="cgs")
    p = yt.plot_2d(ds, var, origin='native')
    p.annotate_timestamp()
    p.annotate_streamlines('b1','b2', density=2, color='white', linewidth=1.2)
    if var == 'rho':
        p.set_zlim(var, 1e-1, 3e0)
    # if var == 'Te': 
    #     p.set_zlim(var, 1e-2, 2e0)
    p.save(f"./figs/{file.split('/')[-1].split('.')[0]}+{var}.png")

# ffmpeg -framerate 12 -pattern_type glob -i ./figs/'*Te.png' -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p Te.mp4 

In [ ]:
# export the points from vtk
import pyvista as pv
import numpy as np
lines = pv.read('fs2d/data/lines.vtk')
# Save points to CSV file
points = lines.points
np.savetxt('fs2d/data/lines_points.csv', points, header=str(points.shape[0]), comments='')

In [ ]:
# prepare cheat
# Filter points where y < 0.25
points_filtered = points[points[:,1] < 0.25]
np.random.shuffle(points_filtered)

# Save filtered points to CSV file
np.savetxt('fs2d/data/cheat.csv', points_filtered, header=str(points_filtered.shape[0]), comments='')
